In [1]:
from typing import List, Optional, Union

import pickle
from functools import partial

import numpy as np
import scipy.stats as stats
from pytest import fixture, importorskip, raises

import aesara.tensor as aet
from aesara import shared
from aesara.configdefaults import config
from aesara.graph.basic import Constant, Variable, graph_inputs
from aesara.graph.fg import FunctionGraph
from aesara.graph.op import get_test_value
from aesara.tensor.basic_opt import ShapeFeature
from aesara.tensor.random.basic import (
    bernoulli,
    beta,
    betabinom,
    binomial,
    categorical,
    cauchy,
    choice,
    dirichlet,
    exponential,
    gamma,
    geometric,
    gumbel,
    halfcauchy,
    halfnormal,
    hypergeometric,
    invgamma,
    laplace,
    logistic,
    lognormal,
    multinomial,
    multivariate_normal,
    nbinom,
    normal,
    pareto,
    permutation,
    poisson,
    polyagamma,
    randint,
    triangular,
    truncexpon,
    uniform,
    vonmises,
    wald,
    weibull,
)
from aesara.tensor.type import iscalar, scalar, tensor

WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Creating a `RandomVariable`

In [2]:
from aesara.tensor.random.op import RandomVariable

class ChiSquareRV(RandomVariable):
    name = "chisquare" # anything else wouldn't work
    ndim_supp = 0
    ndims_params = [0]
    dtype = "floatX"
    _print_name = ("ChiSquare", "\\operatorname{ChiSquare}")


chisquare = ChiSquareRV()

In [3]:
rng = np.random.RandomState(42)
chisquare.rng_fn(rng=rng, df=3, size=5)

array([3.57924635, 2.04721171, 1.86317567, 1.8632062 , 7.67752559])

## Testing

In [7]:
def rv_numpy_tester(rv, *params, **kwargs):
    """Test for correspondence between `RandomVariable` and NumPy shape and
    broadcast dimensions.
    """
    test_fn = kwargs.pop("test_fn", None)

    if test_fn is None:
        name = getattr(rv, "name", None)

        if name is None:
            name = rv.__name__

        test_fn = getattr(np.random, name)

    aesara_res = rv(*params, **kwargs)

    param_vals = [get_test_value(p) if isinstance(p, Variable) else p for p in params]
    kwargs_vals = {
        k: get_test_value(v) if isinstance(v, Variable) else v
        for k, v in kwargs.items()
    }

    if "size" in kwargs:
        kwargs["size"] = get_test_value(kwargs["size"])

    numpy_res = np.asarray(test_fn(*param_vals, **kwargs_vals))

    assert aesara_res.type.numpy_dtype.kind == numpy_res.dtype.kind

    numpy_shape = np.shape(numpy_res)
    numpy_bcast = [s == 1 for s in numpy_shape]
    np.testing.assert_array_equal(aesara_res.type.broadcastable, numpy_bcast)

    aesara_res_val = aesara_res.get_test_value()
    np.testing.assert_array_equal(aesara_res_val.shape, numpy_res.shape)
    
def test_uniform_samples():


    test_low = np.array(10, dtype=config.floatX)
    test_high = np.array(20, dtype=config.floatX)

    rv_numpy_tester(uniform, test_low, test_high)
    rv_numpy_tester(uniform, test_low, test_high, size=[3])

In [5]:
rv_numpy_tester(uniform)

TestValueError: uniform_rv.out has no test value  
Backtrace when that variable is created:

  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2895, in run_cell
    raw_cell, store_history, silent, shell_futures)
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2940, in _run_cell
    return runner(coro)
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/async_helpers.py", line 68, in _pseudo_sync_runner
    coro.send(None)
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3166, in run_cell_async
    interactivity=interactivity, compiler=compiler, result=result)
  File "/Users/larryshamalama/opt/anaconda3/envs/pymc3-dev-py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line

In [6]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon May 17 2021

Python implementation: CPython
Python version       : 3.7.10
IPython version      : 7.22.0

numpy : 1.20.2
scipy : 1.2.1
aesara: 2.0.7

Watermark: 2.2.0

